In [5]:
#Importing required libraries
import numpy as np
import pandas as pd

!pip install geopy

from geopy.geocoders import Nominatim
try:
    import geocoder
except:
    !pip install geocoder
    import geocoder

!pip install beautifulsoup4


import requests
from bs4 import BeautifulSoup

try:
    import folium
except:
    !pip install folium
    import folium
    
from sklearn.cluster import KMeans

     |████████████████████████████████| 112kB 18.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 4.8MB/s ta 0:00:011
     |████████████████████████████████| 829kB 5.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 122kB 6.1MB/s eta 0:00:01


In [6]:
g = geocoder.arcgis('Copenhagen, Denmark')
blr_lat = g.latlng[0]
blr_lng = g.latlng[1]
print("The Latitude and Longitude of Copenhagen is {} and {}".format(blr_lat, blr_lng))

The Latitude and Longitude of Copenhagen is 55.67567000000008 and 12.567560000000071


In [7]:
#Getting the list of localities from Wikipedia site; Suburbs of Copenhagen
neig = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Copenhagen").text

In [8]:
soup = BeautifulSoup(neig, 'html.parser')
#Creating a new list in order to store the neighborhood data
neighborhoodlist = []
for i in soup.find_all('div', class_='mw-category')[0].find_all('a'):
    neighborhoodlist.append(i.text)

#Creating a dataframe from the list
neig_df = pd.DataFrame({"Locality": neighborhoodlist})
neig_df.head()

,Locality
0,Albertslund
1,Avedøre
2,Bagsværd
3,Ballerup
4,Brøndby


In [9]:
#Shape of dataframe neig_df
neig_df.shape

(39, 1)

In [10]:
#Defining a function to get the location of the identified localities from above
def get_location(localities):
    g = geocoder.arcgis('{}, Copenhagen, Denmark'.format(localities))
    get_latlng = g.latlng
    return get_latlng

In [11]:
co_ordinates = []
for i in neig_df["Locality"].tolist():
    co_ordinates.append(get_location(i))
print(co_ordinates)

[[55.6827106498026, 12.485215158980598], [55.67567000000008, 12.567560000000071], [55.67567000000008, 12.567560000000071], [55.67567000000008, 12.567560000000071], [55.67567000000008, 12.567560000000071], [55.67567000000008, 12.567560000000071], [55.75425000000007, 12.568710000000067], [55.705865370235756, 12.465741984268632], [55.67567000000008, 12.567560000000071], [55.67567000000008, 12.567560000000071], [55.67567000000008, 12.567560000000071], [55.681250000000034, 12.588890000000049], [55.722668222308464, 12.490995026644642], [55.79500000000007, 12.536620000000028], [55.74588000000006, 12.425380000000075], [55.69522569398116, 12.545710217862117], [55.67567000000008, 12.567560000000071], [55.66339000000005, 12.467910000000074], [55.681250000000034, 12.588890000000049], [55.69354504611975, 12.542825044385538], [55.650352550056816, 12.620930140317434], [55.77833000000004, 12.590260000000058], [55.77209000000005, 12.505550000000028], [55.696428772349506, 12.538191915314966], [55.675670

In [12]:
#Creating a dataframe from the list of location
co_ordinates_df = pd.DataFrame(co_ordinates, columns=['Latitudes', 'Longitudes'])

In [13]:
#Adding co-ordinates to the neig_df dataframe
neig_df["Latitudes"] = co_ordinates_df["Latitudes"]
neig_df["Longitudes"] = co_ordinates_df["Longitudes"]

In [14]:
neig_df.head()

,Locality,Latitudes,Longitudes
0,Albertslund,55.682711,12.485215
1,Avedøre,55.675670,12.567560
2,Bagsværd,55.675670,12.567560
3,Ballerup,55.675670,12.567560
4,Brøndby,55.675670,12.567560


In [44]:
#Creating a map
blr_map = folium.Map(location=[blr_lat, blr_lng],zoom_start=11)

#adding markers to the map and #marker for Copenhagen
folium.Marker([blr_lat, blr_lng], popup='<i>Copenhagen</i>').add_to(blr_map)

#markers for localities
for latitude,longitude,name in zip(neig_df["Latitudes"], neig_df["Longitudes"], neig_df["Locality"]):
    folium.CircleMarker(
        [latitude, longitude],
        radius=6,
        color='blue',
        popup=name,
        fill=True,
        fill_color='#3186ff'
    ).add_to(blr_map)

blr_map

In [45]:
#Foursquare Credentials
# @hidden_cell
CLIENT_ID = 'V5JVEGFG4DBXKZHZ2VQT44SGEAT2SHI30DVYQHBCAWC30UXD'
CLIENT_SECRET = 'T4KYVGROLUQZD2401UCIJOYWYKRG0L4Z54YA4LHVZ0VG0YRM'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + "CLIENT_ID")
print('CLIENT_SECRET:' + "CLIENT_SECRET")

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


In [46]:
#Getting the top 100 venues in each locality
radius = 2000
LIMIT = 100

venues = []

for lat, lng, locality in zip(neig_df["Latitudes"], neig_df["Longitudes"], neig_df["Locality"]):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']

    for venue in results:
        venues.append((locality, lat, lng, venue['venue']['name'], venue['venue']['location']['lat'], venue['venue']['location']['lng'], venue['venue']['categories'][0]['name']))

In [47]:
venues[0]

('Albertslund',
 55.6827106498026,
 12.485215158980598,
 'Que Pasa',
 55.68415300860668,
 12.485997478267718,
 'Mexican Restaurant')

In [48]:
#Converting the venue list into dataframe
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Locality', 'Latitude', 'Longitude', 'Venue name', 'Venue Lat', 'Venue Lng', 'Venue Category']
venues_df.head()

,Locality,Latitude,Longitude,Venue name,Venue Lat,Venue Lng,Venue Category
0,Albertslund,55.682711,12.485215,Que Pasa,55.684153,12.485997,Mexican Restaurant
1,Albertslund,55.682711,12.485215,Miki's Pizzeria,55.681376,12.484521,Pizza Place
2,Albertslund,55.682711,12.485215,Crossfit Vanløse - Værkstedet,55.684073,12.490756,Gym
3,Albertslund,55.682711,12.485215,Armando's,55.692925,12.489601,Pizza Place
4,Albertslund,55.682711,12.485215,Lagkagehuset,55.687747,12.490913,Bakery


In [49]:
#Number of venues per Locality
venues_df.groupby(['Locality']).count()

,Latitude,Longitude,Venue name,Venue Lat,Venue Lng,Venue Category
Locality,,,,,,
Albertslund,96,96,96,96,96,96
Avedøre,100,100,100,100,100,100
Bagsværd,100,100,100,100,100,100
Ballerup,100,100,100,100,100,100
Brøndby,100,100,100,100,100,100
Buddinge,100,100,100,100,100,100
Charlottenlund,80,80,80,80,80,80
Dyssegård,53,53,53,53,53,53
Ejby (Storkøbenhavn),100,100,100,100,100,100


In [50]:
#Unique categories of the above venues
print('There are {} unique categries.'.format(len(venues_df['Venue Category'])))

There are 3368 unique categries.


In [51]:
#List of categories
print('Total number of unique catefories are {}'.format(len(venues_df['Venue Category'].unique().tolist())))
#First 10 categories
venues_df['Venue Category'].unique().tolist()#[:10]

Total number of unique catefories are 172


['Mexican Restaurant',
 'Pizza Place',
 'Gym',
 'Bakery',
 'Park',
 'Supermarket',
 'Grocery Store',
 'Gym / Fitness Center',
 'Korean Restaurant',
 'Soccer Field',
 'Sports Bar',
 'Shopping Mall',
 'Multiplex',
 'Gastropub',
 'Turkish Restaurant',
 'Concert Hall',
 'Convenience Store',
 'Bar',
 'Ice Cream Shop',
 'Gaming Cafe',
 'Discount Store',
 'Asian Restaurant',
 'Café',
 'Flower Shop',
 'Fish & Chips Shop',
 'Scandinavian Restaurant',
 'Train Station',
 'Thai Restaurant',
 'Fast Food Restaurant',
 'Indian Restaurant',
 'Juice Bar',
 'Plaza',
 'Clothing Store',
 'American Restaurant',
 'Department Store',
 'Light Rail Station',
 'Lake',
 'Burger Joint',
 'Pharmacy',
 'Food & Drink Shop',
 'Hookah Bar',
 'Sporting Goods Shop',
 'Martial Arts School',
 'Smoke Shop',
 'Metro Station',
 'Bus Station',
 'Electronics Store',
 'Apres Ski Bar',
 'Gas Station',
 'Bookstore',
 'Theme Park',
 'Cocktail Bar',
 'Beer Bar',
 'Music Venue',
 'Theme Park Ride / Attraction',
 'Breakfast Spot',
 '

In [52]:
#encoding onehot
blr_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

blr_onehot['Locality'] = venues_df['Locality']

#moving the column, locality to the front
blr_onehot = blr_onehot[ [ 'Locality' ] + [ col for col in blr_onehot.columns if col!='Locality' ] ]
blr_onehot.head()

,Locality,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Apres Ski Bar,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio
0,Albertslund,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Albertslund,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albertslund,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Albertslund,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Albertslund,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
blr_grouped = blr_onehot.groupby(['Locality']).mean().reset_index()
print(blr_grouped.shape)
blr_grouped.head()

(39, 173)


,Locality,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Apres Ski Bar,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio
0,Albertslund,0.0,0.0,0.010417,0.0,0.010417,0.0,0.00,0.0,0.010417,...,0.0,0.0,0.020833,0.010417,0.0,0.0,0.0,0.00,0.00,0.0
1,Avedøre,0.0,0.0,0.000000,0.0,0.000000,0.0,0.01,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.02,0.01,0.0
2,Bagsværd,0.0,0.0,0.000000,0.0,0.000000,0.0,0.01,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.02,0.01,0.0
3,Ballerup,0.0,0.0,0.000000,0.0,0.000000,0.0,0.01,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.02,0.01,0.0
4,Brøndby,0.0,0.0,0.000000,0.0,0.000000,0.0,0.01,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.02,0.01,0.0


In [54]:
#numbers of localities having Italian Restaurants
len(blr_grouped[blr_grouped['Mexican Restaurant'] > 0])

2

In [55]:
blr_mexican = blr_grouped[['Locality', 'Mexican Restaurant']]
blr_mexican.head()

,Locality,Mexican Restaurant
0,Albertslund,0.010417
1,Avedøre,0.000000
2,Bagsværd,0.000000
3,Ballerup,0.000000
4,Brøndby,0.000000


In [56]:
#clustering K-means 
cluster = 3 

#Dataframe for clustering
blr_clustering = blr_mexican.drop(['Locality'], 1)

#running K-means clustering
k_means = KMeans(init="k-means++", n_clusters=cluster, n_init=12).fit(blr_clustering)

#getting the labels for first 10 locality 
print(k_means.labels_[0:10])

[2 0 0 0 0 0 0 0 0 0]


In [57]:
#Creating a blr_mexican dataframe
blr_labels = blr_mexican.copy()

#adding label to blr_labels
blr_labels["Cluster Label"] = k_means.labels_

blr_labels.head()

,Locality,Mexican Restaurant,Cluster Label
0,Albertslund,0.010417,2
1,Avedøre,0.000000,0
2,Bagsværd,0.000000,0
3,Ballerup,0.000000,0
4,Brøndby,0.000000,0


In [58]:
#Merging blr_labels and neig_df dataframes, in order to get the latitude and longitudes for each locality
blr_labels = blr_labels.join(neig_df.set_index('Locality'), on='Locality')
blr_labels.head()

,Locality,Mexican Restaurant,Cluster Label,Latitudes,Longitudes
0,Albertslund,0.010417,2,55.682711,12.485215
1,Avedøre,0.000000,0,55.675670,12.567560
2,Bagsværd,0.000000,0,55.675670,12.567560
3,Ballerup,0.000000,0,55.675670,12.567560
4,Brøndby,0.000000,0,55.675670,12.567560


In [59]:
#Grouping the localities per their Cluster Labels
blr_labels.sort_values(["Cluster Label"], inplace=True)
blr_labels.head()

,Locality,Mexican Restaurant,Cluster Label,Latitudes,Longitudes
19,Jægersborg,0.0,0,55.693545,12.542825
21,Klampenborg,0.0,0,55.778330,12.590260
22,Kongens Lyngby,0.0,0,55.772090,12.505550
23,Lundtofte,0.0,0,55.696429,12.538192
24,Måløv,0.0,0,55.675670,12.567560


In [61]:
#Plot the cluster on map
cluster_map = folium.Map(location=[blr_lat, blr_lng],zoom_start=11)

#marker for Bangalore
folium.Marker([blr_lat, blr_lng], popup='<i>Copenhagen</i>').add_to(cluster_map)

#Getting the colors for the clusters
col = ['red', 'green', 'blue']

#markers for localities
for latitude,longitude,name,clus in zip(blr_labels["Latitudes"], blr_labels["Longitudes"], blr_labels["Locality"], blr_labels["Cluster Label"]):
    label = folium.Popup(name + ' - Cluster ' + str(clus))
    folium.CircleMarker(
        [latitude, longitude],
        radius=6,
        color=col[clus],
        popup=label,
        fill=False,
        fill_color=col[clus],
        fill_opacity=0.3
    ).add_to(cluster_map)
       
cluster_map

In [62]:
#Cluster 1
cluster_1 = blr_labels[blr_labels['Cluster Label'] == 0]
print("There are {} localities in cluster-1".format(cluster_1.shape[0]))
mean_presence_1 = cluster_1['Mexican Restaurant'].mean()
print("The mean occurence of Mexican restaurant in cluster-1 is {0:.2f}".format(mean_presence_1))
cluster_1

There are 37 localities in cluster-1
The mean occurence of Mexican restaurant in cluster-1 is 0.00


,Locality,Mexican Restaurant,Cluster Label,Latitudes,Longitudes
19,Jægersborg,0.0,0,55.693545,12.542825
21,Klampenborg,0.0,0,55.778330,12.590260
22,Kongens Lyngby,0.0,0,55.772090,12.505550
23,Lundtofte,0.0,0,55.696429,12.538192
24,Måløv,0.0,0,55.675670,12.567560
25,Mørkhøj (Gladsaxe),0.0,0,55.715898,12.469359
26,Nærum,0.0,0,55.693790,12.539641
27,Ordrup,0.0,0,55.675670,12.567560
28,Rødovre,0.0,0,55.675670,12.567560
29,Skodsborg,0.0,0,55.697153,12.543660


In [64]:
#Cluster 2
cluster_2 = blr_labels[blr_labels['Cluster Label'] == 1]
print("There are {} localities in cluster-2".format(cluster_2.shape[0]))
mean_presence_2 = cluster_2['Mexican Restaurant'].mean()
print("The mean occurence of Mexican restaurant in cluster-2 is {0:.2f}".format(mean_presence_2))
cluster_2

There are 1 localities in cluster-2
The mean occurence of Mexican restaurant in cluster-2 is 0.01


,Locality,Mexican Restaurant,Cluster Label,Latitudes,Longitudes
32,Solrød Strand,0.013889,1,55.700216,12.4771


In [65]:
#Cluster 3
cluster_3 = blr_labels[blr_labels['Cluster Label'] == 2]
print("There are {} localities in cluster-3".format(cluster_3.shape[0]))
mean_presence_3 = cluster_3['Mexican Restaurant'].mean()
print("The mean occurence of Mexican restaurant in cluster-3 is {0:.2f}".format(mean_presence_3))
cluster_3

There are 1 localities in cluster-3
The mean occurence of Mexican restaurant in cluster-3 is 0.01


,Locality,Mexican Restaurant,Cluster Label,Latitudes,Longitudes
0,Albertslund,0.010417,2,55.682711,12.485215


Conclusion: The above analysis shows that Cluster 1 has many (37) restaurants, while Cluster 2 and 3 has almost no (1) Mexican restaurants. 
This shows that there is a great potential to open a Mexican restaurant, with limited competition, in 2 clusters of the suburbs of Copenhagen
Additionally, it also shows that Cluster 1 is highly competitive with many venues categorized as Mexican Restaurants, thus not recommended to open a new restaurant here.